In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

#Access and recess equation form the Parisian Alfonsines Tables
Hist_table=HTable.read(312,format='dishas')
Hist_table

Arg 2,Tables [12]
deg,
IntegerAndSexagesimal,void128
1 ;,"(1 ; , 00 ; 04) .. (12 ; , 00 ; 50)"
2 ;,"(1 ; , 00 ; 08) .. (12 ; , 01 ; 42)"
3 ;,"(1 ; , 00 ; 12) .. (12 ; , 02 ; 30)"
4 ;,"(1 ; , 00 ; 17) .. (12 ; , 03 ; 20)"
5 ;,"(1 ; , 00 ; 21) .. (12 ; , 04 ; 10)"
6 ;,"(1 ; , 00 ; 25) .. (12 ; , 05 ; 00)"
7 ;,"(1 ; , 00 ; 29) .. (12 ; , 05 ; 50)"
8 ;,"(1 ; , 00 ; 33) .. (12 ; , 06 ; 40)"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(12):
    for j in range(12):
        Test_recomp[i][1][j][1]=Sexagesimal('0;')
        Delta_recomp[i][1][j][1]=Sexagesimal('0;')


In [49]:
for i in range(12):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=2):
        # initial value for dividend
        A1=Hist_table[i][0]
                 
    for j in range(12):
        with set_precision(tmode=TruncatureMode.TRUNC, pmode=2):
            # second values for dividend
            A2=Hist_table[0][1][j][0]
            prop=A1*A2/Historical('14;24')
         
        # computation of the proportion
        with set_precision(tmode=TruncatureMode.TRUNC, pmode=1):
            
            Test_recomp[i][1][j][1]=1*prop*u.degree
            hist=Hist_table[i][1][j][1]
            Delta_recomp[i][1][j][1]=60*(hist-prop)

In [50]:
# residuals visualisation and statistical summaries 1
A=Delta_recomp.to_pandas()
(A.mean().mean(), A.std().std())

(-0.513888888888889, 3.2959289364080546)

In [51]:
# residuals visualisation and statistical summaries 2
A.astype(float).style.background_gradient(axis=None)

,1 ;,2 ;,3 ;,4 ;,5 ;,6 ;,7 ;,8 ;,9 ;,10 ;,11 ;,12 ;
1 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-10.000000,0.000000
4 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,0.000000
6 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,-2.000000,0.000000
7 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-16.000000,0.000000,0.000000
8 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000
9 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10 ;,0.000000,0.000000,0.000000,-6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-38.000000,0.000000,0.000000


In [52]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Prop for min .csv')
B.to_csv('Prop for min.zip', index=False, compression=compression_opts)